In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_set_util = DataSetUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/air/air_quality_beijing_1_site.csv', sep=',')
df = df.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
df.dropna(inplace=True)
data = data_set_util.clean_dataset(df)
data.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,4.4
1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,4.7
2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,5.6
3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,3.1
4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,2.0


In [5]:
data.describe()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,31876.000000,31876.000000,31876.000000,31876.000000,31876.000000,31876.000000,31876.000000,3.187600e+04,31876.000000,31876.000000,31876.000000
mean,81.883436,109.134854,22.663822,59.020862,1257.172324,120.897715,54.202537,1.043531e+03,3.235387,0.068108,1.717926
std,80.460506,93.706644,367.548951,37.091708,1223.586402,1803.538176,1091.904425,5.663287e+03,13.666100,0.836693,1.205326
min,3.000000,2.000000,0.571200,2.000000,100.000000,0.214200,-9575.000000,9.859000e+02,-35.300000,0.000000,0.000000
25%,22.000000,37.000000,3.000000,30.000000,500.000000,8.000000,3.500000,1.003300e+03,-8.000000,0.000000,0.900000
50%,58.000000,86.000000,8.000000,53.000000,800.000000,42.000000,14.900000,1.011200e+03,4.000000,0.000000,1.400000
75%,114.000000,154.000000,21.000000,82.000000,1500.000000,83.000000,23.500000,1.020000e+03,15.700000,0.000000,2.200000
max,713.000000,948.000000,47124.000000,290.000000,10000.000000,88893.000000,28425.000000,1.012125e+06,28.500000,46.400000,11.200000


In [6]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,PM2.5(t-1),PM10(t-1),SO2(t-1),NO2(t-1),CO(t-1),O3(t-1),TEMP(t-1),PRES(t-1),DEWP(t-1),RAIN(t-1),...,PM10(t),SO2(t),NO2(t),CO(t),O3(t),TEMP(t),PRES(t),DEWP(t),RAIN(t),WSPM(t)
1,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,...,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,4.7
2,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,...,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,5.6
3,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,...,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,3.1
4,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,...,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,2.0
5,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,...,5.0,18.0,18.0,400.0,66.0,-2.2,1025.6,-19.6,0.0,3.7


In [7]:
df_forecats_columns = reframed.loc[:,'PM2.5(t)':'WSPM(t)'].columns

In [8]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    num_fuzzy_rules = {
        "window": [],
        "qtd": []
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'PM2.5(t-1)':'WSPM(t-1)']
            df_test = ttest.loc[:,'PM2.5(t-1)':'WSPM(t-1)']
            df_original = ttest.loc[:,'PM2.5(t)':'WSPM(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            #teste print fuzzy rules 
            sum_rules = 0
            for m in models:
                sum_rules = sum_rules + len(m)
                #print(len(m))
            
            #print(sum_rules)
            num_fuzzy_rules["window"].append(ct)
            num_fuzzy_rules["qtd"].append(sum_rules)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    rules = pd.DataFrame(num_fuzzy_rules)
    
    return measures, rules

## Sliding Window: MIMO KPCA-NSFTS

In [9]:
num_components_pca = 2
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [11]:
kpca_result, rules =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 1062
--------------------
training window 2124
--------------------
training window 3186
--------------------
training window 4248
--------------------
training window 5310


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 6372
--------------------
training window 7434
--------------------
training window 8496
--------------------
training window 9558
--------------------
training window 10620
--------------------
training window 11682
--------------------
training window 12744
--------------------
training window 13806
--------------------
training window 14868
--------------------
training window 15930
--------------------
training window 16992
--------------------
training window 18054
--------------------
training window 19116
--------------------
training window 20178
--------------------
training window 21240
--------------------
training window 22302
--------------------
training window 23364


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 24426
--------------------
training window 25488
--------------------
training window 26550
--------------------
training window 27612
--------------------
training window 28674
--------------------
training window 29736
--------------------
training window 30798


In [11]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

kpca_result.to_csv (r'all_windows_kpca_mo_ensfts_uci_air_quality_beijing_1_site_.csv', index = False, header=True)

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [12]:
print("Statistics MIMO KPCA-NSFTS(test): ")
pd.set_option('display.max_rows', None)
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5(t),21.269,13.596,37.999,0.871,13.609,0.076,9.390,5.589,18.810,0.076,4.952,0.016
1,PM10(t),33.795,20.857,36.840,0.783,13.632,0.091,12.194,6.301,13.823,0.126,3.379,0.022
2,SO2(t),5.250,3.183,25.943,0.809,11.306,0.080,4.232,2.645,9.640,0.104,3.135,0.023
3,NO2(t),13.354,8.916,23.695,0.791,9.352,0.095,3.195,1.818,10.262,0.078,2.298,0.015
4,CO(t),349.124,226.501,24.504,0.826,10.204,0.084,222.183,142.277,10.918,0.093,2.726,0.017
5,O3(t),1229.635,477.918,1680.267,0.722,24.783,0.109,4669.159,1987.282,7979.203,0.450,15.364,0.046
6,TEMP(t),39.594,15.195,inf,0.402,14.706,0.102,205.605,74.551,NaN,2.206,17.962,0.112
7,PRES(t),1.310,1.039,0.102,0.834,0.051,0.060,2.462,2.061,0.201,0.652,0.101,0.084
8,DEWP(t),1.410,0.969,inf,0.908,9.102,0.066,0.350,0.249,NaN,0.048,6.209,0.011
9,RAIN(t),0.539,0.090,NaN,-0.073,NaN,NaN,0.746,0.107,NaN,0.619,NaN,NaN


In [13]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_1_site_with_std.csv', index = False, header=True)

In [12]:
avg_rules = round(statistics.mean(rules.loc[:,'qtd']),3)
avg_rules

113.5

In [14]:
#final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_1_site_2.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [13]:
num_components_pca = 6
order_fts_model = 1
npart = 70
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [14]:
pca_result,rules =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 1062
--------------------
training window 2124
--------------------
training window 3186
--------------------
training window 4248


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 5310


...

--------------------
training window 15930
--------------------
training window 16992
--------------------
training window 18054
--------------------
training window 19116
--------------------
training window 20178
--------------------
training window 21240


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 22302
--------------------
training window 23364


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 24426
--------------------
training window 25488
--------------------
training window 26550
--------------------
training window 27612
--------------------
training window 28674
--------------------
training window 29736
--------------------
training window 30798


In [19]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

pca_result.to_csv (r'all_windows_pca_mo_ensfts_uci_air_quality_beijing_1_site.csv', index = False, header=True)

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 


print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5(t),25.290,17.884,53.323,0.827,19.318,0.090,11.023,7.576,29.165,0.071,7.747,0.017
1,PM10(t),38.711,26.631,50.861,0.732,18.090,0.103,14.678,9.947,19.821,0.124,5.471,0.022
2,SO2(t),7.302,5.251,59.592,0.571,23.532,0.117,5.428,4.059,30.011,0.352,9.198,0.036
3,NO2(t),16.330,12.082,31.960,0.685,12.969,0.115,4.797,3.397,11.015,0.136,2.750,0.018
4,CO(t),450.005,323.856,37.140,0.700,16.215,0.111,246.456,180.323,16.064,0.179,6.759,0.024
5,O3(t),1599.564,1112.070,19789.868,0.545,34.177,0.136,6422.449,4812.417,93580.321,0.890,21.101,0.075
6,TEMP(t),37.679,16.961,inf,0.657,16.493,0.121,193.533,82.857,NaN,0.254,19.152,0.037
7,PRES(t),2.267,1.696,0.168,0.702,0.084,0.119,1.022,0.633,0.062,0.223,0.031,0.043
8,DEWP(t),2.509,1.925,inf,0.717,15.294,0.114,1.227,0.969,NaN,0.214,11.065,0.036
9,RAIN(t),0.666,0.387,NaN,-0.501,NaN,NaN,0.880,0.534,NaN,2.188,NaN,NaN


In [20]:
final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_1_site_with_std.csv', index = False, header=True)

In [ ]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_1_site_2.csv', index = False, header=True)

In [24]:
# pd.set_option('display.max_rows', None)
# final_measures_pca

In [15]:
avg_rules = round(statistics.mean(rules.loc[:,'qtd']),3)
avg_rules

316.2